# SPR 2026 - Phi-3.5 Mini 3.8B (Zero-Shot Classification)

## ✅ Compatível com competições que exigem Internet OFF

**Este notebook usa Phi-3.5 Mini 3.8B do Kaggle Models.**

### Configuração:

1. **NÃO precisa de internet** - funciona offline
2. Adicione o modelo: `microsoft/phi-3.5` → Variação `mini-instruct`
3. Settings → Accelerator → **GPU T4 x2** ou **P100**
4. Run All

### Vantagens:
- **Muito eficiente** (~8GB VRAM)
- **Rápido** - otimizado para inferência
- **Multilingual** - bom em português

---

In [ ]:
# =============================================================================
# SPR 2026 - PHI-3.5 MINI ZERO-SHOT (OFFLINE)
# =============================================================================
# REQUER: Modelo "microsoft/phi-3.5" (variação mini-instruct) adicionado
# =============================================================================

import os
import numpy as np
import pandas as pd
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

print("="*60)
print("SPR 2026 - Phi-3.5 Mini 3.8B Zero-Shot (OFFLINE)")
print("="*60)

# ==== CONFIGURAÇÕES ====
SEED = 42
DATA_DIR = '/kaggle/input/spr-2026-mammography-report-classification'

# Paths para modelos Kaggle
MODEL_PATHS = [
    '/kaggle/input/phi-3.5/transformers/mini-instruct/1',
    '/kaggle/input/phi-3.5/transformers/mini-instruct',
    '/kaggle/input/phi-3-5/transformers/mini-instruct/1',
    '/kaggle/input/phi3.5/transformers/mini-instruct/1',
]

MODEL_PATH = None
for path in MODEL_PATHS:
    if os.path.exists(path):
        MODEL_PATH = path
        break

if MODEL_PATH is None:
    print("\n⚠️ Modelo não encontrado. Datasets disponíveis:")
    for item in os.listdir('/kaggle/input'):
        print(f"  - {item}")
        try:
            for sub in os.listdir(f'/kaggle/input/{item}')[:3]:
                print(f"      └── {sub}")
        except:
            pass
    raise FileNotFoundError("Adicione o modelo microsoft/phi-3.5 (mini-instruct) ao notebook!")

USE_GPU = torch.cuda.is_available()
device = 'cuda' if USE_GPU else 'cpu'

print(f"✓ Modelo: {MODEL_PATH}")
print(f"✓ GPU disponível: {USE_GPU}")
print(f"✓ Device: {device}")

In [ ]:
# ==== CARREGAR DADOS ====
print("\n[1/4] Carregando dados...")
train = pd.read_csv(f'{DATA_DIR}/train.csv')
test = pd.read_csv(f'{DATA_DIR}/test.csv')
print(f"    Train: {train.shape} | Test: {test.shape}")

print("\nDistribuição das classes no treino:")
print(train['target'].value_counts().sort_index())

In [ ]:
# ==== CARREGAR MODELO ====
print("\n[2/4] Carregando Phi-3.5 Mini 3.8B...")

tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(
    MODEL_PATH,
    torch_dtype=torch.float16 if USE_GPU else torch.float32,
    device_map='auto' if USE_GPU else None,
    trust_remote_code=True,
)

if not USE_GPU:
    model = model.to(device)

model.eval()
print("    ✓ Modelo carregado!")
print(f"    ✓ VRAM estimada: ~8GB")

In [ ]:
# ==== PROMPT TEMPLATE (Phi-3.5 format) ====
# Phi-3.5 usa formato <|user|> ... <|end|> <|assistant|>

def build_prompt(report):
    return f"""<|user|>
Você é um radiologista especialista em mamografia.
Classifique o seguinte laudo de mamografia na categoria BI-RADS correta.

Categorias BI-RADS:
0 - Inconclusivo, necessita exames adicionais
1 - Negativo, sem achados
2 - Achados benignos
3 - Provavelmente benigno
4 - Suspeito de malignidade
5 - Altamente suspeito de malignidade
6 - Malignidade confirmada por biópsia

Laudo:
{report[:2000]}

Responda APENAS com o número da categoria BI-RADS (0, 1, 2, 3, 4, 5 ou 6):
<|end|>
<|assistant|>"""

def classify_report(report, max_length=1024):
    """Classifica um laudo usando o LLM."""
    prompt = build_prompt(report)
    
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=max_length)
    inputs = {k: v.to(device) for k, v in inputs.items()}
    
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=5,
            temperature=0.1,
            do_sample=False,
            pad_token_id=tokenizer.eos_token_id,
        )
    
    response = tokenizer.decode(outputs[0][inputs['input_ids'].shape[1]:], skip_special_tokens=True)
    
    # Extrair número da resposta
    for char in response.strip():
        if char.isdigit() and int(char) in range(7):
            return int(char)
    
    return 2  # Fallback

In [ ]:
# ==== INFERÊNCIA ====
print("\n[3/4] Classificando laudos do teste...")

predictions = []
for idx, row in tqdm(test.iterrows(), total=len(test), desc="Classificando"):
    pred = classify_report(row['text'])
    predictions.append(pred)

test['target'] = predictions

print("\nDistribuição das predições:")
print(test['target'].value_counts().sort_index())

In [ ]:
# ==== SALVAR SUBMISSION ====
print("\n[4/4] Salvando submission...")

submission = test[['id', 'target']].copy()
submission.to_csv('submission.csv', index=False)

print("\n" + "="*60)
print("✅ SUBMISSION SALVA: submission.csv")
print("="*60)
print(submission.head(10))